Import dataset

In [2]:
import json

# Specify the path to your JSON file
json_file_path = 'Metaphor_CoT_explanation_total.json'

# Open the JSON file and load its contents into a dictionary
with open(json_file_path, 'r') as json_file:
    data_dict = json.load(json_file)

# Now, 'data_dict' contains the contents of the JSON file as a dictionary


TypeError: ignored

In [3]:
# modify the data so that it could be converted to huggingface dataset
dataset = []
for i in data_dict:
  dataset.append(i["CoT"])

In [4]:
dataset[0]

{'instruction': "The input text contains a text called 'literature_context', which contains a sentence. Identify whether the sentence is metaphoric or literal and analyze the metaphor if metaphoric.",
 'input': '\n        literature_context: Sentence: The storm clouds gathered, heavy with the weight of their burdens\n        ',
 'output': 'Sentence: The storm clouds gathered, heavy with the weight of their burdens.\n\nContext: Weather forecast for an impending storm.\n\nExplanation: The content of the sentence is metaphoric. The storm clouds gathering and being described as "heavy with the weight of their burdens" is a metaphor. It is comparing the physical presence of the storm clouds to a metaphorical representation of burden or heaviness. The metaphor emphasizes the intensity and impending severity of the storm, suggesting that it carries significant emotional or metaphorical weight.'}

In [10]:
model = '3B' #'7B' # Pick your poison

if model == '7B':
    model_name = ("togethercomputer/RedPajama-INCITE-Base-7B-v0.1","togethercomputer/RedPajama-INCITE-Base-7B-v0.1")
    run_name = 'redpj7B-lora-int8-alpaca'
    dataset = 'johnrobinsn/alpaca-cleaned'
    peft_name = 'redpj7B-lora-int8-alpaca'
    output_dir = 'redpj7B-lora-int8-alpaca-results'
else: #3B
    model_name = ("togethercomputer/RedPajama-INCITE-instruct-3B-v1","togethercomputer/RedPajama-INCITE-instruct-3B-v1")
    run_name = 'redpj3B-lora-int8-logiqa'
    dataset = 'Metaphor_CoT_explanation_total'
    peft_name = 'redpj3B-lora-int8-Metaphor_CoT'
    output_dir = 'redpj3B-lora-int8-Metaphor_CoT-results'

model_name[1],dataset,peft_name,run_name

('togethercomputer/RedPajama-INCITE-instruct-3B-v1',
 'lucasmccabe/logiqa',
 'redpj3B-lora-int8-logiqa',
 'redpj3B-lora-int8-logiqa')

In [5]:
def install_dependencies():
    !pip install -Uqq  git+https://github.com/huggingface/peft.git
    !pip install -Uqq transformers datasets accelerate bitsandbytes
    !pip install -Uqq wandb

# uncomment the following line to install the required dependencies
install_dependencies()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [7]:
from datasets import Dataset
# Convert the list of dictionaries to a Hugging Face Dataset
dict_data = {key: [item[key] for item in dataset] for key in dataset[0]}
huggingface_dataset = Dataset.from_dict(dict_data)

print(huggingface_dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 2638
})


In [8]:
huggingface_dataset[0]

{'instruction': "The input text contains a text called 'literature_context', which contains a sentence. Identify whether the sentence is metaphoric or literal and analyze the metaphor if metaphoric.",
 'input': '\n        literature_context: Sentence: The storm clouds gathered, heavy with the weight of their burdens\n        ',
 'output': 'Sentence: The storm clouds gathered, heavy with the weight of their burdens.\n\nContext: Weather forecast for an impending storm.\n\nExplanation: The content of the sentence is metaphoric. The storm clouds gathering and being described as "heavy with the weight of their burdens" is a metaphor. It is comparing the physical presence of the storm clouds to a metaphorical representation of burden or heaviness. The metaphor emphasizes the intensity and impending severity of the storm, suggesting that it carries significant emotional or metaphorical weight.'}

Monitoring Weights and biases

In [25]:
report_to = "wandb" # "none"

if report_to != "none":
    import wandb
    wandb.login()
#04b000cbb7e17ddd83e391aa3996b53c7f064da7

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [26]:
wandb.init(project=run_name,config={
    "model": model_name[1],
    "dataset":"metaphor_CoT"
})

wandb: Currently logged in as: jl6298. Use `wandb login --relogin` to force relogin


Loading tokenizer

In [11]:
from transformers import AutoTokenizer

print("Loading tokenizer for model: ", model_name[1])
tokenizer = AutoTokenizer.from_pretrained(model_name[1],add_eos_token=True)
tokenizer.pad_token_id = 0

Loading tokenizer for model:  togethercomputer/RedPajama-INCITE-instruct-3B-v1


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [12]:
tokenizer.add_special_tokens({'eos_token':'<eos>'})
print('eos_token_id:',tokenizer.eos_token_id)

eos_token_id: 50277


In [13]:
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data in the alpaca dataset

def tokenize(prompt, tokenizer,add_eos_token=True):
    result = tokenizer(
        prompt+"<eos>",  # add the end-of-stream token
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"],
        "attention_mask": result["attention_mask"],
    }

In [16]:
tokenizer("""instruction': "The input text contains a text called 'literature_context', which contains a sentence. Identify whether the sentence is metaphoric or literal and analyze the metaphor if metaphoric.'input': '\n        literature_context: Sentence: The storm clouds gathered, heavy with the weight of their burdens\n        ','output': 'Sentence: The storm clouds gathered, heavy with the weight of their burdens.\n\nContext: Weather forecast for an impending storm.\n\nExplanation: The content of the sentence is metaphoric. The storm clouds gathering and being described as "heavy with the weight of their burdens" is a metaphor. It is comparing the physical presence of the storm clouds to a metaphorical representation of burden or heaviness. The metaphor emphasizes the intensity and impending severity of the storm, suggesting that it carries significant emotional or metaphorical weight""")

{'input_ids': [50103, 5295, 346, 510, 3280, 2505, 4428, 247, 2505, 1925, 686, 22478, 1177, 64, 8882, 1383, 534, 4428, 247, 6197, 15, 15108, 1419, 1880, 253, 6197, 310, 27294, 280, 390, 22436, 285, 12106, 253, 27294, 604, 27294, 280, 2464, 5423, 5295, 686, 187, 50270, 22478, 1177, 64, 8882, 27, 20580, 566, 27, 380, 9902, 16173, 13037, 13, 5536, 342, 253, 2801, 273, 616, 32274, 187, 50270, 10606, 9252, 5295, 686, 29710, 566, 27, 380, 9902, 16173, 13037, 13, 5536, 342, 253, 2801, 273, 616, 32274, 15, 187, 187, 5856, 27, 28323, 16923, 323, 271, 40911, 9902, 15, 187, 187, 1672, 45525, 27, 380, 2600, 273, 253, 6197, 310, 27294, 280, 15, 380, 9902, 16173, 16778, 285, 1146, 2529, 347, 346, 37893, 342, 253, 2801, 273, 616, 32274, 3, 310, 247, 27294, 15, 733, 310, 10941, 253, 3520, 3361, 273, 253, 9902, 16173, 281, 247, 27294, 474, 6779, 273, 7977, 390, 3573, 1632, 15, 380, 27294, 35520, 253, 7133, 285, 40911, 12147, 273, 253, 9902, 13, 7738, 326, 352, 15814, 1534, 8991, 390, 27294, 474, 2801], 

Since the dataset already has the prompting format, we only need to combine them into one string.

In [17]:
def generate_prompt(data_point):
    if data_point["instruction"]:
        return f"""{data_point["instruction"]}

{data_point["input"]}

{data_point["output"]}"""


In [18]:
generate_prompt(huggingface_dataset[5])

"The input text contains a text called 'literature_context', which contains a sentence. Identify whether the sentence is metaphoric or literal and analyze the metaphor if metaphoric.\n\n\n        literature_context: Sentence: The sun was a blazing ball of fire in the sky\n        \n\nSentence: The sun was a blazing ball of fire in the sky.\n\nContext: The speaker is describing the appearance of the sun in the sky.\n\nExplanation: This sentence is metaphoric. The content compares the sun to a blazing ball of fire, suggesting that the sun is emitting intense heat and bright light just like a fire. The metaphor emphasizes the intense and powerful nature of the sun."

In [19]:
VAL_SET_SIZE = 520  # we set aside about 1/5 items from our dataset for validation during training

train_val = huggingface_dataset.train_test_split(
    test_size=VAL_SET_SIZE, shuffle=True, seed=42
)
train_data = train_val["train"]
val_data = train_val["test"]

In [20]:
train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))
val_data = val_data.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))

Map:   0%|          | 0/2118 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

In [21]:
train_data

Dataset({
    features: ['instruction', 'input', 'output', 'input_ids', 'attention_mask'],
    num_rows: 2118
})

In [22]:
from transformers import AutoModelForCausalLM

print("Loading model for model: ", model_name[0])

model = AutoModelForCausalLM.from_pretrained(
    model_name[0],
    load_in_8bit=True,
    device_map="auto",
)

Loading model for model:  togethercomputer/RedPajama-INCITE-instruct-3B-v1


config.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [23]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# Define LoRA Config
lora_config = LoraConfig(
 r= 8,
 lora_alpha=16,
 target_modules=["query_key_value"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.CAUSAL_LM
)

# prepare int-8 model for training
model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,621,440 || all params: 2,778,485,760 || trainable%: 0.09434779323828531


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Setup trainer

In [31]:
import transformers
eval_steps = 100
save_steps = 100
logging_steps = 20

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        num_train_epochs=3,
        learning_rate=3e-4,
        logging_steps=logging_steps,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=eval_steps,
        save_steps=save_steps,
        output_dir=output_dir,
        report_to=report_to if report_to else "none",
        save_total_limit=3,
        load_best_model_at_end=True,
        push_to_hub=False,
        auto_find_batch_size=True
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [32]:
trainer.train()

Step,Training Loss,Validation Loss
100,0.762900,0.747315
200,0.694500,0.697581
300,0.645300,0.681151
400,0.610300,0.669256
500,0.638100,0.657478
600,0.586600,0.656377
700,0.598100,0.650120


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain 

TrainOutput(global_step=795, training_loss=0.6605182899619049, metrics={'train_runtime': 4986.5024, 'train_samples_per_second': 1.274, 'train_steps_per_second': 0.159, 'total_flos': 2.585726899716096e+16, 'train_loss': 0.6605182899619049, 'epoch': 3.0})

In [33]:
peft_name = 'redpj3B-lora-int8-Metaphor_CoT'
trainer.model.save_pretrained(peft_name)
tokenizer.save_pretrained(peft_name)

('redpj3B-lora-int8-Metaphor_CoT/tokenizer_config.json',
 'redpj3B-lora-int8-Metaphor_CoT/special_tokens_map.json',
 'redpj3B-lora-int8-Metaphor_CoT/tokenizer.json')

Evaluation step

In [35]:
def generate_prompt2(data_point):
    # sorry about the formatting disaster gotta move fast
        return f"""Below is an 'A is-a B' sentence. Please identify if A and B are metaphoric use case in the sentence and answer with 'yes' or 'no'.
            ### sentence:
            {data_point["sentence"]}

            ### A:
            {data_point["A"]}

            ### B:
            {data_point["B"]}

            ### answer:
            """
dataset_ = "Joanne/Metaphors_and_Analogies"
# Load dataset from the hub
from datasets import load_dataset
data_ = load_dataset(dataset_, 'Pairs_Cardillo_random_split')

def inference (text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  tokenizer.pad_token= tokenizer.eos_token
  input_ids = tokenizer.encode(
    text,
    return_tensors="pt",
    truncation=True,
    max_length=max_input_tokens
  )
  # Generate
  device=model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens,
    pad_token_id=tokenizer.eos_token_id
  )
  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens = True)
  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]
  return generated_text_answer


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [71]:
data_["train"][12]["sentence"]

"The writer's job is a lonely drive."

In [73]:
inference("""Is the sentence "The conversation was a hushed whisper." a use case of metaphor? explain why it is or why not.""",trainer.model,tokenizer,max_output_tokens=100).lower()

'\n\nthe conversation was a hushed whisper.\n\nexplanation:\n\nthe content of the sentence is metaphorical. it uses the metaphor of a hushed whisper to describe the conversation. the metaphor suggests that the conversation was quiet and low-key, as if it were being whispered rather than spoken aloud. the use of the word "hushed" adds'

In [76]:
json_list = []
def evaluation_prompt(sentence):
  return """Is the sentence '{sentence}' a use case of metaphor? answer with Yes or No."""
for datapoint in data_['train']:
  output = inference(evaluation_prompt(datapoint["sentence"]),trainer.model,tokenizer,max_output_tokens=1)
  json_list.append({"input":datapoint["sentence"],"output":output})
json_list_total_string = json.dumps(json_list, indent=4)
with open("fintuned_redpajama_metaphor_output.json", 'w+') as file:
        file.write(json_list_total_string)

In [78]:
model_not_finetuned = AutoModelForCausalLM.from_pretrained(
    model_name[0],
    load_in_8bit=True,
    device_map="auto",
)

In [79]:
inference("""Is the sentence "The conversation was a hushed whisper." a use case of metaphor? explain why it is or why not.""",model_not_finetuned,tokenizer,max_output_tokens=100).lower()

'\n\nlabel: \nno\n\n\ntitle: a great book for the beginner\nreview: i bought this book for my daughter who is a beginner. she is very happy with it. it is a great book for the beginner.\nis the review positive or negative?\n\nlabel: \npositive\n\n\ntitle: a great book for the beginner\n'

Loading finetuned model from disk

In [ ]:
model_path = "redpj3B-lora-int8-Metaphor_CoT"

# Load the model
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Load the tokenizer
loaded_tokenizer = AutoTokenizer.from_pretrained(model_path)